# Sem Pormpt

In [2]:
from openai import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import SystemMessage
import numpy as np
from IPython.display import display, Markdown

In [3]:
client = OpenAI()

In [4]:
#Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)
    
#Text Splitter
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)


#Verificando se foi extraido corretamente
print(split_texts[0])

#Embbeding e Vector Store
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(
    documents=split_texts,
    embedding=embedding_model
)

#Retriver
retriever = vector_store.as_retriever()
# Preparar o contexto
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)
#Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
        }
    ]
)


#Resposta
print(completion.choices[0].message.content)


page_content='Adicionar verificação de configuração dos arquivos .xml 
 
PROCEDIMENTO (XXXX) CENTRAL 2.0 
iles de hardware e software da Central 2.0 verificando o perfeito funcionamento da mesma. Na 
sequência, o cadastramento da central no DEVICE MANAGER para controle de estoque.  
A configuração com a aplicação do cliente a ser instalada, junto com informações de rede 
fecham o procedimento (XXXX). 
 
Para informações sobre imagens explodidas da central 2.0 prosseguir ao procedimento forçar 
(XXXX). 
Para informações sobre a montagem da central 2.0 prosseguir ao procedimento (XXXX). 
. 
1.​Pré-teste Versão de Hardware 2.0 
 
Verificar se o GATEWAY está conectado a Raspberry.' metadata={'producer': 'Skia/PDF m136 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'files/Configuração e teste - Central SensorGW.pdf', 'file_path': 'files/Configuração e teste - Central SensorGW.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': 'Configuração e teste - Central SensorGW

# Com Prompt

In [5]:
# Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)
    
# Text Splitter
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_texts = text_splitter.create_documents(texts)

# Embedding e Vector Store
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(
    documents=split_texts,
    embedding=embedding_model
)

# Retriever
retriever = vector_store.as_retriever()

# Pegar os documentos relevantes baseado na pergunta
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)

# Preparar o contexto
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)

# Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", 
         "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado, em formato de passo a passo.\n\n{context}"
         },
        {"role": "user", 
        "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
         } 
    ]
)

# Resposta
display(Markdown(completion.choices[0].message.content))


Para configurar a central para o cliente, siga os passos detalhados abaixo:

1. Acesse o terminal PUTTY ou qualquer outro software de terminal para se conectar à central.

2. Execute o comando sensorgw appConfig para iniciar a configuração da central:
   ```
   sensorgw appConfig
   Default server configuration? (y or n): y
   Client WebApp: "TESTE"
   Client Login:
   Client Password:
   CheckConnection:
   CheckModem:
   Carrier: "operadora"
   ```

3. Execute o comando sync para atualizar as configurações.
   ```
   sync
   ```

4. Pare a execução do processo com o comando sensorgw stop.
   ```
   sensorgw stop
   ```

5. Agora, vamos configurar o Hamachi. Continue no PUTTY.

6. Pare o serviço do Hamachi com o comando:
   ```
   /etc/init.d/logmein-hamachi stop
   ```

7. Remova quaisquer configurações anteriores do Hamachi com o comando:
   ```
   rm -Rf /var/lib/logmein-hamachi/
   ```

8. Inicie o serviço do Hamachi com o comando:
   ```
   /etc/init.d/logmein-hamachi start
   ```

9. Faça login no Hamachi digitando:
   ```
   hamachi login
   ```

10. Defina o nome da central para o Hamachi com o comando:
    ```
    hamachi set-nick "CLIENTE"
    ```

11. Vincule a conta do Hamachi com o comando:
    ```
    hamachi attach eduardosleal@mcasistemas.com.br
    ```

12. Conecte-se à conta do Hamachi utilizando:
    ```
    hamachi do-join 158-893-675
    Senha = mcaUser
    ```

13. Após configurar o Hamachi, verifique a configuração dos arquivos .xml.

Pronto! A central foi configurada para enviar dados para o portal do cliente de acordo com as instruções fornecidas. Certifique-se de seguir cada etapa detalhadamente para garantir a configuração correta da central.

# Com Hyde

In [6]:
# Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Dividir o texto
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_texts = text_splitter.split_documents(docs)

# Criar embeddings e armazenar em um vetor
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents=split_texts, embedding=embedding_model)
retriever = vector_store.as_retriever()

# Definir a pergunta
pergunta = "Como é faço para configurar a central para o cliente?"
hyde_model = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125", 
    temperature=0
)
hyde_prompt = f"""
Responda à seguinte pergunta de forma detalhada e clara. Seja preciso e forneça uma explicação passo a passo:

Pergunta: {pergunta}

Utilize as informações abaixo para criar uma resposta completa:
"""
resposta_hipotetica = hyde_model.invoke([SystemMessage(content=hyde_prompt)])

# Criar embedding da resposta hipotética para busca mais precisa
resposta_hipotetica_embedding = embedding_model.embed_query(resposta_hipotetica.content)

# Criar embedding da pergunta original para busca mais precisa
pergunta_embedding = embedding_model.embed_query(pergunta)

# Garantir que ambos os embeddings sejam numpy arrays
pergunta_embedding = np.array(pergunta_embedding)
resposta_hipotetica_embedding = np.array(resposta_hipotetica_embedding)

# Combinar os embeddings da pergunta original e da resposta hipotética
combined_embedding = (pergunta_embedding + resposta_hipotetica_embedding) / 2

# Buscar documentos com o embedding combinado
retrieved_docs = vector_store.similarity_search_by_vector(combined_embedding)

# Criar o contexto com os documentos recuperados
context_list = []
for doc in retrieved_docs:
    context_list.append(doc.page_content)

# Juntar os contextos recuperados em uma única string
context = "\n\n".join(context_list)

# Gerar a resposta final com base no contexto recuperado
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado e em formato de passo a passo.\n\n{context}"},
        {"role": "user","content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"}
    ]
)

# Exibir a resposta gerada
display(Markdown(completion.choices[0].message.content))


Para configurar a central para o cliente, siga os passos abaixo:

1. **Acesse o PUTTY**: Utilize o software PUTTY para estabelecer uma conexão com a central. Insira o IP ou nome de host da central na tela de conexão e clique em "Open".

2. **Execute os comandos de configuração inicial**:
    - No PUTTY, insira o comando `sensorgw appConfig` para iniciar a configuração da aplicação.
    - Quando questionado sobre a configuração padrão do servidor, responda 'y' para sim.
    - Preencha os dados solicitados, como o nome do Cliente (Client WebApp) e a operadora de telefonia (Carrier).
    - Execute o comando `sync` para sincronizar as configurações.
    - Pare a aplicação com o comando `sensorgw stop`.

3. **Configure o HAMACHI**:
    - Pare o serviço HAMACHI com o comando `/etc/init.d/logmein-hamachi stop`.
    - Remova quaisquer configurações anteriores do HAMACHI utilizando o comando `rm -Rf /var/lib/logmein-hamachi/`.
    - Inicie o serviço HAMACHI com o comando `/etc/init.d/logmein-hamachi start`.
    - Realize o login no HAMACHI com o comando `hamachi login`.
    - Defina o nome da central no HAMACHI com o comando `hamachi set-nick "CLIENTE"`.
    - Vincule a conta do HAMACHI com o comando `hamachi attach eduardosleal@mcasistemas.com.br`.
    - Conecte-se à conta do HAMACHI usando `hamachi do-join 158-893-675` e insira a senha mcaUser.

4. **Atribua as informações de rede para IP Dinâmico**:
    - Certifique-se de que a central está configurada para obter um IP Dinâmico (DHCP).
    - Acesse o navegador e insira o IP da central para acessar as configurações.
    - Use as credenciais fornecidas (Login: teste, Senha: teste).

5. **Teste a autonomia da bateria**:
    - Desconecte a fonte de energia da central e verifique se ela continua funcionando apenas com a bateria.

6. **Teste o botão ON/OFF**:
    - Teste o botão de ligar/desligar da central, verificando se um clique desliga a central e se pressionar e manter pressionado o botão força o desligamento.

Após seguir esses passos, a central estará devidamente configurada para o cliente e pronta para operação.